# USU WUDC Trial Debate Allocation Algorithm
Developed by Viran Weerasekera (2017-18)

In [ ]:
import gurobipy as grb
import pandas as pd
import numpy as np
import math
import datetime

def allocate(swing):
    global times
    times = {}
    alltimes = {1:"9:00am",2:"9:30am",3:"12:30pm",4:"2:00pm"}#,5:"2:00pm",6:"3:30pm",7:"5:00pm",8:"6:30pm"}
    #morningslots = [1,2]
    #afternoonslots = [3,4]
    
    # Import data
    global vetoes
    vetoes = {}
    global speakers
    speakers = []
    
    csv = pd.read_csv("USU Worlds Trials 2019 (Responses) - Form responses 1.csv",index_col=0)

    for index,row in csv.iterrows():
        if row[5] != "Adjudicating ONLY":
            speakers.append(row[0])
        #No speaker position or availability vetoes in Worlds
        
    #Form Feasible Team Set (FTS)
    print("{} speakers to allocate{}".format(len(speakers),", swings added" if len(speakers) % 8 >=0 and swing else ""))
    print("-----")
    global fts
    fts = {}
    index = 0
    for speaker1 in speakers:
        for speaker2 in speakers:
            if speaker1 != speaker2:
                    fts[index] = [speaker1,speaker2]
                    index += 1
    if swing:
        #Swing Construction
        if len(speakers) % 8 >= 2:
            fts[index] = ["Swing A","Swing B"]
            index += 1
            speakers.append("Swing A")
            speakers.append("Swing B")
        if len(speakers) % 8 >= 4:
            fts[index] = ["Swing C","Swing D"]
            index += 1
            speakers.append("Swing C")
            speakers.append("Swing D")
        if len(speakers) % 8 >= 6:
            fts[index] = ["Swing E","Swing F"]
            index += 1
            speakers.append("Swing E")
            speakers.append("Swing F")
            
    #Calculate how many timeslots are required
    alltimeslots = (1,2,3,4,5,6,7,8)
    debatesreqd = 2*int(np.floor(len(speakers)/8))
    timeslots = alltimeslots[:debatesreqd]
    morningslots = alltimeslots[:debatesreqd/2]
    afternoonslots = []
    for slot in timeslots:
        if slot not in morningslots:
            afternoonslots.append(slot)
    
    halves = ["Opening Half","Closing Half"]
    
    #Generate Costs
    costs = {}
    for key,time in alltimes.iteritems():
        if key in timeslots:
            times[key] = time

    for key,team in fts.iteritems():
        for slot in timeslots:
            for half in halves:
                costs[key,slot,half] = 0
                    
        for speaker in team:
            rlvconflicts = []
            for conflictobject in conflicts:
                if speaker in conflictobject:
                    rlvconflicts.append(conflictobject)
        for rlvconflict in rlvconflicts:
            for spk in rlvconflict:
                if spk in team:
                    for slot in timeslots:
                        for half in halves:
                            costs[key,slot,half] = grb.GRB.INFINITY
                    
    #Optimisation
    a = grb.Model("2018 WUDC Trial Allocator")
    x = {}
    for key,value in costs.iteritems():
        x[key] = a.addVar(vtype = grb.GRB.INTEGER, obj = value, name = "x_{}".format(key))

    #Exactly 4 teams in each timeslot
    for slot in timeslots:
        a.addConstr(sum(x[team,slot,half] for team in fts.keys() for half in halves) == 4)
        for half in halves:
            a.addConstr(sum(x[team,slot,half] for team in fts.keys()) == 2)

    #Each team to no more than 1 timeslot
    for key,team in fts.iteritems():
        a.addConstr(sum(x[key,slot,half] for slot in timeslots for half in halves) <= 1)


    for speaker in speakers:
        affectedTeams = []
        for key,team in fts.iteritems():
            if speaker in team:
                affectedTeams.append(key)
        # Each speaker gets one slot in each half
        for half in halves:
            a.addConstr(sum(x[team,slot,half] for team in affectedTeams for slot in timeslots) == 1)
        
        #Each speaker gets one debate in the morning and afternoon groups
        a.addConstr(sum(x[team,slot,half] for team in affectedTeams for slot in morningslots for half in halves) == 1)
        a.addConstr(sum(x[team,slot,half] for team in affectedTeams for slot in afternoonslots for half in halves) == 1)
        
    a.modelSense = grb.GRB.MINIMIZE
    a.optimize()
  
    allocatedSpeakers = []
    unallocatedSpeakers = []
    print("-----")
    print("USU WORLDS TRIALS 2018 - DEBATE ALLOCATION")
    #print("Speakers in each team are listed in random order.")
    #print("Teams are listed in essentially random order.")
    print("***Number of triallists is not a multiple of 4 - {} speakers will not be allocated.".format(len(speakers) % 8) if len(speakers) % 8 > 0 else "Allocation generated at {}".format(datetime.datetime.now()))
    print("-----")
    for key,value in costs.iteritems():
        if x[key].x != 0:
            print("Team {} ({}) {} in Debate {} ({})".format(key[0],key[2],fts[key[0]],key[1],times[key[1]]))
            for speaker in fts[key[0]]:
                allocatedSpeakers.append(speaker)

    for speaker in speakers:
        if speaker not in allocatedSpeakers:
            unallocatedSpeakers.append(speaker)
    print("-----")
    print("Successfully allocated {} out of {} speakers across {} debates").format(len(allocatedSpeakers),len(speakers),debatesreqd)
    print("Unallocated Speakers: {}".format(", ".join(unallocatedSpeakers)) if len(unallocatedSpeakers) > 0 else "No unallocated speakers")
    a.write("2018 WUDC Allocation.rlp")
    print("-----" if len(unallocatedSpeakers)>0 else "")
    print("Unallocated Speaker Information:" if len(unallocatedSpeakers)>0 else "")
    for s in unallocatedSpeakers:
        report(s)

conflicts = []
def addConflict(conflicter,conflictee):
    conflicts.append([conflicter,conflictee])
    
def report(speaker):
    rlv = []
    for conflict in conflicts:
        if conflict[0] == speaker:
            rlv.append(conflict[1])
        elif conflict[1] == speaker:
            rlv.append(conflict[0])
    if len(rlv) == 0:
        print("{} has no conflicts.".format(speaker))
    else:
        conflictstring = ", ".join(rlv)
        print("{} cannot debate with: {}".format(speaker, conflictstring))

In [ ]:
#1. Add Conflicts
#2. Allocate
allocate(False)